```sentences_NH/engbook37``` is an input file being referenced here, but I don't have, nor do I have a script that generates it. 

Using the Vecalign script vecalign.py (partially modified) from https://github.com/thompsonb/vecalign, the code takes as input the originary files containing the sentences, the overlap files, the files containing the embeddings. The output of the alignment is a list of indexes pairs and a float number with the compute of the alignment cost.

The modification of the vecalign.py file consists in generating a .TXT file containing the output of the alignment.

In [1]:
# # Use latest conda
# !conda update conda -y
# # Create conda environment
# !conda create  --force -y --name vecalign python=3.7
# # Activate new environment
# !source `conda info --base`/etc/profile.d/conda.sh # See: https://github.com/conda/conda/issues/7980
# !conda activate vecalign
# # Install required packages
# !conda install -y -c anaconda cython
# !conda install -y -c anaconda numpy
# !pip install mcerp 

In [2]:
## generate overlaps file using the eng and lat sentences in the sentences_NH folder
# !python overlap.py -i sentences_NH/engbook37 -o sentences_NH/engoverlaps37.eng -n 10
# !python overlap.py -i sentences_NH/latbook37 -o sentences_NH/latoverlaps37.lat -n 10

#we do not use shell calls any more. We import the overlap utility and call the go function. 
#this function is normally called whenever the user performs the command above
#Inside the overlap utility the go function is a redundant copy of make_overlap(), we call that 
#without retaing go for maintainability. 
#we call make_overlap with arguments: inputfile, outputfile, numberoverlaps
#TODO: make parameters dynamic!
#   replaced by conten in next cell
# from utils import overlap
# overlap.make_overlap('sentences_NH/engbook37', 'sentences_NH/engoverlaps37.eng', 10)
# overlap.make_overlap('sentences_NH/latbook37', 'sentences_NH/latoverlaps37.lat', 10)

In [1]:
from utils import overlap
from utils.dp_core import *
from setuptools import distutils  #TODO required?
from tqdm import tqdm
import os
from utils import vecalign

#TODO: make parameters dynamic
#TODO: discuss with Laura; where are the source files?
#       the target file is created by the overlap utility; does not need to exist.
#       BUT, we need to know what it is used for!

for i in tqdm(range(1,38)): 
    overlap.make_overlap(source = 'data/intermediate/books/engbook'+str(i), 
                     target = 'data/intermediate/books/engoverlaps'+str(i)+'.eng', 
                     num_overlaps = 10
                     )
    overlap.make_overlap(source = 'data/intermediate/books/latbook'+str(i), 
                     target = 'data/intermediate/books/latoverlaps'+str(i)+'.lat', 
                     num_overlaps = 10
                     )

100%|██████████████████████████████████████████████████████████████████████████████████| 37/37 [00:02<00:00, 12.43it/s]


In [4]:
# pip install sentence_transformers

In [2]:
## import the S-BERT Python library sentence_transformer
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

C:\Workdir\MyApps\Python_VENV\geoparsing_naturalhistory-main\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Workdir\MyApps\Python_VENV\geoparsing_naturalhistory-main\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
##CONFIG:
#when set to True this will redo the labse files even if they exists.
#when set to False, LABSE files will be retained, a new model won't be made.
renew_labse_files = False

basedir = 'data/intermediate/books'
lang1 = 'eng'
lang2 = 'lat'

files = os.listdir(basedir)
# Filtering only the files.
files = [f for f in files if os.path.isfile(basedir+'/'+f) and f.strip().endswith(lang1)]

#prepare a typle of embeddings with per tuple the Source and Target
all_embeddings = []

# for i in range(1,38):    
for i in files:    

    ## get the eng overlap file
    # path = "/Users/u0154817/OneDrive - KU Leuven/Desktop/sentences_NH/engoverlaps"+str(i)+'.eng'
    path = basedir+'/'+i
    print(path)
    with open(path, "r", encoding="utf8") as engfile:
        overlap_sentences_eng = []
        for line in engfile: ## create a list of the overlap eng sentences
            overlap_sentences_eng.append(line)
    
    ## get the lat overlap file
    path = path.replace('engoverlaps', 'latoverlaps')
    path = path.replace('.eng', '.lat')
    print(path)
    with open(path, "r", encoding="utf8") as latfile:
        overlap_sentences_lat = []
        for line in latfile:  ## create a list of the overlap lat sentences
            overlap_sentences_lat.append(line)
    ## generate the name of the LABSE files: 
    j = i.split('.')[0]
    j = j.replace('engoverlaps', '').strip()
    src_embedded_filename = 'overlaps.'+lang1+'.'+j+'.LaBSE.emb'
    tgt_embedded_filename = 'overlaps.'+lang2+'.'+j+'.LaBSE.emb'
    # if the files exist, do not override them:
    engoverlap_missing = not os.path.exists('data/intermediate/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    latoverlap_missing = not os.path.exists('data/intermediate/labse_embeddings/overlaps.lat.'+str(j)+'.LaBSE.emb')
    ## generate the embeddings of the eng and lat overlap sentences (conditionally)
    if renew_labse_files or engoverlap_missing: 
        print(f'making english embeddings: {len(overlap_sentences_eng)}')
        overlaps_embeddings_eng_LaBSE = model.encode(overlap_sentences_eng)
        overlaps_embeddings_eng_LaBSE.tofile('data/intermediate/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+src_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')
    if renew_labse_files or latoverlap_missing:
        print(f'making latin embeddings: {len(overlap_sentences_lat)}')
        overlaps_embeddings_lat_LaBSE = model.encode(overlap_sentences_lat)
        overlaps_embeddings_lat_LaBSE.tofile('data/intermediate/labse_embeddings/overlaps.lat.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+tgt_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')



data/intermediate/books/engoverlaps1.eng
data/intermediate/books/latoverlaps1.lat
making english embeddings: 626


In [11]:
import os 
dir = os.getcwd()
os.path.join(dir, 'data', 'intermediate', 'books', 'engbook10')

'c:\\Workdir\\MyApps\\Python_VENV\\geoparsing_naturalhistory-main\\data\\intermediate\\books\\engbook10'

In [22]:
##TODO: here you'll need to create a forloop that iterates over all labse_embeddings files. 
# categorize them in latin and english ambeddings
# link them by the number in their name
# then run the linked versions in the vecalign_custom method.
#get the booksfolder: 

#Creates vecalign output in: \data\intermediate\alignments

embeddingsfolder = os.path.join(dir, 'data', 'intermediate', 'labse_embeddings')
bookids = [book.split('.')[2] for book in os.listdir(embeddingsfolder) if '.eng.' in book]
for book in tqdm(bookids): 
    i = os.path.join(dir, 'data', 'intermediate', 'books', 'engbook'+book)
    path = os.path.join(dir, 'data', 'intermediate', 'books', 'latbook'+book)
    src_embedded_filename = 'data/intermediate/labse_embeddings/overlaps.eng.'+book+'.LaBSE.emb'
    tgt_embedded_filename = 'data/intermediate/labse_embeddings/overlaps.lat.'+book+'.LaBSE.emb'
    vecalign.vecalign_custom(
            src=i, 
            tgt=path, 
            src_embed=('data/intermediate/books/engoverlaps'+book+'.eng', src_embedded_filename), 
            tgt_embed=('data/intermediate/books/latoverlaps'+book+'.lat', tgt_embedded_filename) 
        )

['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '4', '5', '6', '7', '8', '9']


  0%|          | 0/36 [00:00<?, ?it/s]2024-06-17 17:02:17,342  WARNI  expected an embedding size of 1024, got 768
2024-06-17 17:02:17,430  WARNI  expected an embedding size of 1024, got 768
c:\Workdir\MyApps\Python_VENV\geoparsing_naturalhistory-main\utils\dp_utils.py:635: RuntimeWarning: invalid value encountered in cast
  stack[depth]['a_b_costs'], stack[depth]['b_offset'] = make_sparse_costs(stack[depth]['v0'], stack[depth]['v1'],
  3%|▎         | 1/36 [00:00<00:24,  1.43it/s]2024-06-17 17:02:18,036  WARNI  expected an embedding size of 1024, got 768


[0, 1]:[0, 1]:0.302867
[2]:[2]:0.555216
[3]:[3]:0.518855
[4]:[4]:0.530393
[5]:[5]:0.432686
[6]:[6]:0.464447
[7]:[7, 8]:0.334580
[8]:[9]:0.450746
[9]:[10, 11]:0.403053
[10]:[12]:0.542799
[11]:[13]:0.665190
[12]:[14]:0.489490
[13, 14]:[15]:0.442630
[15, 16]:[16]:0.479696
[17]:[17]:0.279546
[18, 19]:[18]:0.333856
[20]:[19]:0.325738
[21, 22]:[20]:0.446207
[23]:[21]:0.355677
[24]:[22]:0.325428
[25]:[23]:0.654092
[26]:[24]:0.379653
[27]:[25]:0.256552
[28]:[26]:0.366630
[29]:[27]:0.377843
[30]:[28, 29]:0.541046
[31]:[30, 31]:0.335261
[32]:[32]:0.185542
[33]:[33]:0.419116
[34]:[34]:0.595111
[35]:[35]:0.649812
[36]:[36, 37]:0.335594
[37]:[38, 39]:0.483006
[38]:[40]:0.594795
[39]:[41]:0.543655
[40]:[42, 43]:0.702460
[41]:[44]:0.515172
[42]:[45]:0.534093
[43]:[46]:0.522721
[44]:[47]:0.454565
[45]:[48]:0.586640
[46]:[49]:0.251729
[47, 48]:[50, 51]:0.219986
[49]:[52]:0.328137
[50]:[53]:0.386186
[51]:[54]:0.454766
[52]:[55, 56]:0.316123
[53]:[57]:0.617928
[54, 55]:[58]:0.423329
[56]:[59]:0.428451
[5

2024-06-17 17:02:18,131  WARNI  expected an embedding size of 1024, got 768
  6%|▌         | 2/36 [00:01<00:26,  1.28it/s]2024-06-17 17:02:18,824  WARNI  expected an embedding size of 1024, got 768


[0]:[0, 1]:0.651727
[1, 2, 3]:[2]:0.458607
[4]:[]:0.000000
[5]:[]:0.000000
[6]:[]:0.000000
[7]:[]:0.000000
[8, 9, 10]:[3]:0.362556
[11]:[4]:0.518058
[12]:[5, 6, 7]:0.333400
[13]:[8, 9]:0.649054
[14]:[10]:0.543251
[15]:[11]:0.303485
[16, 17]:[12]:0.251593
[18]:[13]:0.340285
[19]:[14]:0.206318
[20]:[15, 16]:0.297523
[21]:[17]:0.293955
[22]:[18]:0.410706
[23]:[19]:0.223056
[24]:[20]:0.475325
[25]:[21]:0.361386
[26]:[22]:0.433848
[27, 28, 29]:[23]:0.436015
[30, 31, 32]:[24]:0.650352
[33, 34]:[25]:0.483531
[35, 36]:[26]:0.366826
[37]:[27]:0.309106
[38]:[28]:0.521854
[39]:[29]:0.324257
[40]:[30]:0.310136
[41]:[31]:0.448062
[42]:[32]:0.338188
[43]:[33, 34]:0.458223
[44]:[35]:0.492646
[45]:[36]:0.417229
[46]:[37]:0.273015
[47]:[38]:0.480846
[48]:[39]:0.645679
[49]:[40]:0.313880
[50]:[41]:0.255314
[51, 52, 53]:[42]:0.273111
[54]:[43, 44, 45]:0.451052
[55]:[46]:0.421173
[56]:[47]:0.383170
[57]:[48]:0.583026
[58]:[49, 50]:0.314816
[59]:[51]:0.728708
[60]:[52]:0.426193
[61]:[53]:0.641815
[62]:[54,

...

[0, 1]:[0]:0.422049
[2]:[1]:0.477829
[3]:[2]:0.551619
[4]:[3]:0.267997
[5, 6]:[4]:0.419393
[7, 8, 9]:[5]:0.660617
[10]:[]:0.000000
[11, 12, 13]:[6]:0.335628
[14, 15]:[7]:0.540119
[16]:[8]:0.209218
[17]:[9, 10, 11]:0.251810
[18]:[12]:0.483550
[19, 20]:[13]:0.407395
[21, 22, 23]:[14]:0.397910
[24, 25, 26]:[15]:0.273643
[27]:[16, 17]:0.363859
[28]:[18]:0.467012
[29]:[19]:0.386800
[30]:[20]:0.302157
[31]:[21]:0.477095
[32]:[22]:0.308849
[33]:[23]:0.467266
[34]:[24]:0.319898
[35]:[25]:0.366605
[36]:[26]:0.344182
[37]:[27]:0.268502
[38]:[28]:0.445634
[39]:[29, 30]:0.292267
[40]:[31]:0.348926
[41]:[32]:0.258002
[42]:[33]:0.392722
[43]:[34]:0.264886
[44]:[35]:0.270072
[45]:[36]:0.247011
[46]:[37, 38]:0.240034
[47]:[39]:0.290453
[48]:[40]:0.261070
[49]:[41]:0.294475
[50]:[42, 43]:0.257671
[51]:[44, 45]:0.385593
[52]:[46]:0.528899
[53]:[47, 48]:0.274654
[54]:[49]:0.268940
[55]:[50, 51]:0.367144
[56]:[52]:0.272030
[57]:[53, 54]:0.546930
[58]:[55]:0.359226
[59]:[56]:0.215909
[60]:[57]:0.324448
[61

 97%|█████████▋| 35/36 [00:21<00:00,  1.75it/s]

[0]:[0]:0.306717
[1]:[1]:0.322127
[2]:[2]:0.367278
[3, 4]:[3]:0.472940
[5]:[4]:0.409357
[6]:[5]:0.530245
[7]:[6]:0.348112
[8]:[7]:0.363429
[9]:[8]:0.399419
[10]:[9]:0.319473
[11]:[10]:0.616919
[12]:[11]:0.523146
[13]:[12]:0.691585
[14]:[13]:0.303237
[15]:[14, 15]:0.446001
[16]:[16, 17]:0.324199
[17]:[18]:0.667911
[18]:[19]:0.513188
[19]:[20]:0.473626
[20]:[21, 22]:0.305217
[21, 22]:[23]:0.341435
[23]:[24]:0.699628
[24, 25, 26]:[25]:0.317540
[27]:[26, 27]:0.534159
[28]:[28]:0.636142
[29, 30]:[29]:0.334844
[31]:[30, 31]:0.367696
[32]:[32, 33]:0.439447
[33]:[34, 35]:0.331493
[34]:[36]:0.227571
[35]:[37]:0.610108
[36]:[38]:0.539322
[37]:[39]:0.495577
[38]:[40]:0.349909
[39]:[41]:0.279002
[40]:[42]:0.586714
[41]:[43]:0.552316
[42]:[44]:0.400409
[43]:[45]:0.377930
[44]:[46]:0.565925
[45]:[47]:0.306021
[46]:[48]:0.514867
[47]:[49, 50]:0.357627
[48]:[51]:0.379790
[49]:[52]:0.406369
[50]:[53]:0.521055
[51]:[54]:0.312784
[52]:[55]:0.258386
[53]:[56]:0.309586
[54]:[57]:0.688697
[55]:[58]:0.452063

2024-06-17 17:02:39,048  WARNI  expected an embedding size of 1024, got 768
2024-06-17 17:02:39,099  WARNI  expected an embedding size of 1024, got 768
100%|██████████| 36/36 [00:22<00:00,  1.60it/s]

[0]:[0]:0.377744
[1]:[1]:0.476745
[2]:[2]:0.572824
[3]:[3]:0.627795
[4]:[4]:0.597447
[5]:[5]:0.733785
[6]:[6]:0.361598
[7]:[7]:0.412336
[8]:[8]:0.443833
[9]:[9]:0.419976
[10]:[10]:0.451890
[11]:[11]:0.395371
[12]:[12]:0.438339
[13]:[13]:0.462780
[14]:[14]:0.425244
[15]:[15]:0.314544
[16]:[16]:0.353783
[17]:[17]:0.497135
[18]:[18]:0.294343
[19]:[19, 20]:0.297909
[20]:[21]:0.362895
[21]:[22]:0.297381
[22]:[23]:0.399231
[23, 24]:[24]:0.318638
[25]:[25]:0.625052
[26]:[26, 27]:0.431414
[27]:[28]:0.467362
[28]:[29]:0.443168
[29]:[30]:0.616124
[30]:[31]:0.517574
[31]:[32, 33]:0.339558
[32]:[34]:0.336919
[33]:[35]:0.505257
[34]:[36]:0.306945
[35]:[37]:0.403061
[36]:[38]:0.340391
[37, 38, 39]:[39]:0.213504
[40]:[40, 41]:0.247120
[41]:[42]:0.602326
[42]:[43]:0.373189
[43]:[44]:0.290956
[44]:[45]:0.247140
[45]:[46]:0.240316
[46]:[47]:0.409471
[47]:[48]:0.507270
[48]:[49]:0.425896
[49]:[50]:0.419630
[50]:[51]:0.271435
[51]:[52]:0.478940
[52]:[53]:0.408716
[53]:[54]:0.388739
[54]:[55]:0.511023
[55]

In [9]:
# ## calculate the sentence similarity using the sentence files, the overlap files, and the sentence embeddings
# !python vecalign.py --alignment_max_size 8 --src sentences_NH/engbook37 --tgt sentences_NH/latbook37 \
#    --src_embed sentences_NH/engoverlaps37.eng sentences_NH/overlaps.eng.37.LaBSE.emb  \
#    --tgt_embed sentences_NH/latoverlaps37.lat sentences_NH/overlaps.lat.37.LaBSE.emb

In [10]:
# vecalign(
#     src='sentences_NH/engbook37', 
#     tgt='sentences_NH/latbook37', 
#       src_embed and tgt_embed have to become tuples: 
#       first is a text file, second is a binary embeddings file
#     src_embed='sentences_NH/engoverlaps37.eng sentences_NH/overlaps.eng.37.LaBSE.emb', 
#     tgt_embed='sentences_NH/latoverlaps37.lat sentences_NH/overlaps.lat.37.LaBSE.emb'
#     )